
# AES (Advanced Encrypton Standard) #

[Leírás](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.197.pdf)

DES (Feistel network alapú) leváltására

Spec. Rijndael network
substitution-permutation network

Kulcs méretek: 128, 192, 256 bits (van egyéb Rijndael network is, de ezek kerültek be a szabványba)

Blokkok:

$\left(\begin{array}{rrrr}
k_0 & k_4 & k_8    & k_{12} \\
k_1 & k_5 & k_9    & k_{13} \\
k_2 & k_6 & k_{10} & k_{14} \\
k_3 & k_7 & k_{11} & k_{15}
\end{array}\right)\quad k_i\in \mathbb{F}_{2^8}$

Kulcsok méretének megfelelően 10, 12, 14 lépésben kapjuk meg a titkosított blokkot


### Rijndael véges test ###

$\mathbb{F}_2[x]/_{x^8+x^4+x^3+x+1}$

In [0]:
Z2x.<x> = PolynomialRing(GF(2))
f = x^8+x^4+x^3+x+1; factor(f)
RF.<y> = Z2x.quotient_ring(f); RF.is_field()
RF.cardinality()
1/(y+1)
1/RF('y+1')


és kell a rotációhoz is egy gyűrű

In [0]:
factor(x^8+1) #nem lesz test
FR.<z> = Z2x.quotient_ring(x^8+1); FR.is_field()
[1/z^i for i in range(8)] #szerencsere minden forgatásnak van inverze


### Rijndael S-Box ###
(spec. Nyberg S-box)

byte $\to$ byte transzformáció (cryptoanalízis ellen)

Ha $c$ eleme a Rijndael testnek akkor a transzformáció: elem inverzének (RF-ben) affin transzformációját vesszük a következő mátrixal és vektorral

In [0]:
print("Mátrix (M):")
M = Matrix(GF(2),8,lambda i,j: (i-j) % 8 < 5); pretty_print(M)
print("Vektor (V):")
V = Matrix(GF(2), [0,1,1,0,0,0,1,1]); pretty_print(V) #0x63


S-Box: $c \mapsto M*tobin(c^{-1}) + V$

vagy azaz a korábban definiált strukturák segítségével:

In [0]:
m = z^4+z^3+z^2+z+1
v = z^6+z^5+z+1
c = y^7+y^4+y^3+y #0x9A
def SBox(c):
    return RF(FR(1/c)*m + v)
SBox(c) #0xB8 = x^7+x^5+x^4+x^3
SBox(1) #0x01 -> 0x7C
FR(y^(-7))


Inverz S-Box
* V additív inverze $\checkmark$
* M inverze $\checkmark$
* Rijandel testben inverz $\checkmark$

de megadható affin transformációval is

Inv-SBox: $c\mapsto \frac{1}{toRF(M^{-1}*c + M^{-1}*V)}$

In [0]:
print("Mátrix az inverzhez:")
invM = M.inverse();
pretty_print(invM)
print("Vektor az inverzhez:")
invV = invM*V.transpose();
pretty_print(invV.transpose())

In [0]:
def InvSBox(c):
    return 1/RF((FR(c)-v)*1/m)
mul([c == InvSBox(SBox(c)) for c in RF if c != 0]) #should be one


## Kulcs készítése minden körhöz ##

A szükséges $k$ körhöz $k+1$ 'round key' készítése a bemeneti kulcs méretének megfelelően, azaz 11,13 vagy 15 db kell.

In [0]:
Nb = 6
Ok = [[RF.random_element() for i in range(4)] for j in range(Nb)]; #original key 192 bit - something random
print('Original %d (%d*4*8bit = %d*32bit = 6w) bit (random) key' % (32*Nb, Nb, Nb))
pretty_print(Matrix(Ok))

def make_round_keys(key):
    rks = key
    N = len(key) # number of input words
    for i in range(N,N+8):
        nk = rks[i-N]
        pk = rks[i-1]
        if i % N == 0:
            nk = [nk[j] + SBox(pk[(j+1) % 4]) + y^int(i/N) for j in range(4)]
        elif N > 6 and i % N == 4:
            nk = [nk[j] + SBox(pk[j]) for j in range(4)]
        else:
            nk = [nk[j] + pk[j] for j in range(4)]
        rks.append(nk)
    return rks

print('Round keys:'); i = 1
round_keys = make_round_keys(Ok)
pretty_print(Matrix(round_keys))


## Shift rows ##

A bemenet egy 32 bites blokk, ahogy ez itt szokott lenni

In [0]:
ri = [[RF.random_element() for i in range(4)] for j in range(4)]
print('Input')
pretty_print(Matrix(ri))

def shiftrows(d):
    r = []
    for i in range(4):
        r.append([d[i][(j+i) % 4] for j in range(4)])
    return r

print('Result')
pretty_print(Matrix(shiftrows(ri)))


## AES ##

1. AddRoundKey
2. n-2 kör: SubBytes, ShiftRows, MixClumns, AddRoundKey
3. SubBytes, ShiftRows, AddroundKey

### Feladatok ###

1. Írjunk függvényt, amely $\mathbb{Z_2}[x]$ beli poinomot ír fel egy 0 és 1-et tartalmazó string formájában (a változó is bemenet).
2. Írjunk függvényt, amely egy bitsorozatot alakít a két elemű test feletti polinommá (a változó is bemenet).
3. Ellenőrizzük le, hogy a rotációhoz tartozó gyűrőben az $y^i$ tényleg rotálja a polinomokhoz tartozó bitvektorokat.
3. Készíts táblázatot (dictionary/hashmap) az S-Box számolásához.
4. Készíts táblázatot az inverz S-Box számolásához.
5. Írjuk meg a kulcs kiterjesztést bináris adatokra (lehetőleg "bináris" műveleteket használva).
6. Írjuk meg a ShiftRows függvény inverzét.
7. Írjuk meg a ShiftRows függvényt és inverzét bináris adatokra. (Tekinthető 2db feladatnak.)